In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay


In [ ]:
TARGET_SIZE = (64, 64)   # keep feature size manageable
DATA_DIR = "/kaggle/input/plantvillage-dataset/color"


In [ ]:
def load_data(directory):
    X = []  # features
    y = []  # labels
    
    for label in os.listdir(directory):
        label_path = os.path.join(directory, label)
        
        if not os.path.isdir(label_path):
            continue
        
        for file in os.listdir(label_path):
            if file.lower().endswith(('.jpg', '.png')):
                img_path = os.path.join(label_path, file)
                
                # Read image
                img = cv2.imread(img_path)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                
                # Resize
                img = cv2.resize(img, TARGET_SIZE)
                
                # Flatten 
                vector = img.flatten()
                
                X.append(vector)
                y.append(label)
    
    return np.array(X), np.array(y)


In [ ]:
X, y = load_data(DATA_DIR)

print("Feature shape:", X.shape)
print("Labels shape:", y.shape)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
dummy = DummyClassifier(strategy="most_frequent")
dummy.fit(X_train_scaled, y_train)

y_dummy = dummy.predict(X_test_scaled)
dummy_acc = accuracy_score(y_test, y_dummy)

print("Dummy Baseline Accuracy:", dummy_acc)


In [ ]:
rf = RandomForestClassifier(
    n_estimators=200,
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train, y_train)   # NOTE: RF does NOT need scaled data

y_pred = rf.predict(X_test)
y_train_pred = rf.predict(X_train)
train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_pred)

print("Random Forest Training Accuracy:", train_acc)
print("Random Forest Accuracy:", test_acc)

In [ ]:
cm = confusion_matrix(y_test, y_pred, labels=rf.classes_)

disp = ConfusionMatrixDisplay(
    confusion_matrix=cm,
    display_labels=rf.classes_
)

plt.figure(figsize=(10, 10))
disp.plot(xticks_rotation=90)
plt.title("Random Forest Confusion Matrix")
plt.show()
